In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import re

# Dataset untuk chatbot pendengaran
intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": [
                "hello",
                "hi",
                "hai",
                "good morning",
                "good afternoon",
                "good evening",
                "morning",
                "other greetings"
            ],
            "responses": [
                "Hello! Welcome to the Hearity application. We do not understand your question yet. Please type 'Info' to see quick questions."
            ]
        },
        {
            "tag": "main_menu",
            "patterns": [
                "info",
                "menu",
                "options",
                "information",
                "what are the menus",
                "help"
            ],
            "responses": [
                "Welcome to our service! Please type the following number to get an answer from the set of questions we have provided:\n1. What is hearing test?\n2. What is hearing loss?\n3. Other Information"
            ]
        },
        {
            "tag": "hearing_test",
            "patterns": [
                "1",
                "what is hearing test",
                "hearing test",
                "hearing test process",
                "how does hearing test work"
            ],
            "responses": [
                "A hearing test is a clinical evaluation that measures an individual's ability to hear sounds at various frequencies and volumes to assess hearing health."
            ]
        },
        {
            "tag": "hearing_loss",
            "patterns": [
                "2",
                "what is hearing loss",
                "hearing loss",
                "loss of hearing",
                "hearing impairment"
            ],
            "responses": [
                "Hearing loss is a partial or total inability to hear sounds, which can result from age, noise exposure, genetics, or other medical conditions."
            ]
        },
        {
            "tag": "submenu_other",
            "patterns": [
                "3",
                "other information"
            ],
            "responses": [
                "Here are other questions:\n4. What is hearing aid?\n5. What's the maximum decibels for ears?\n6. More Information"
            ]
        },
        {
            "tag": "hearing_aid",
            "patterns": [
                "4",
                "what is hearing aid",
                "hearing aid",
                "assistive hearing device"
            ],
            "responses": [
                "A hearing aid is a small electronic device worn in or behind the ear that amplifies sound to assist individuals with hearing loss."
            ]
        },
        {
            "tag": "decibel_info",
            "patterns": [
                "5",
                "maximum decibels",
                "safe decibel level",
                "safe sound limits"
            ],
            "responses": [
                "Prolonged exposure to sounds above 85 decibels can cause hearing damage; sounds at 120 decibels can lead to immediate harm."
            ]
        },
        {
            "tag": "more_questions",
            "patterns": [
                "6",
                "more information"
            ],
            "responses": [
                "Here are other questions:\n7. How to prevent hearing loss if working in a noisy place?\n8. How to take care of ears?\n9. Why is it important to check ear health?\n10. More options"
            ]
        },
        {
            "tag": "prevention",
            "patterns": [
                "7",
                "how to prevent hearing loss",
                "prevent hearing loss",
                "hearing loss prevention"
            ],
            "responses": [
                "Keep ears clean and dry, avoid inserting objects into the ear canal, protect ears from loud noises, and schedule regular check-ups with an audiologist."
            ]
        },
        {
            "tag": "ear_protection",
            "patterns": [
                "8",
                "how to protect ears",
                "ear protection",
                "protect hearing",
                "care ear"
            ],
            "responses": [
                "Use hearing protection such as earplugs or earmuffs, and take regular breaks from noisy environments."
            ]
        },
        {
            "tag": "ear_health",
            "patterns": [
                "9",
                "why check ear health",
                "importance of ear health",
                "ear health"
            ],
            "responses": [
                "Regular ear health checks can detect hearing problems early, prevent further damage, and ensure effective communication and overall well-being."
            ]
        },
        {
            "tag": "other",
            "patterns": [
                "10",
                "other options"
            ],
            "responses": [
                "Sorry, we are only a chatbot service. For more information, please visit the nearest hospital. You can check through our feature to find the nearest hospital."
            ]
        }
    ]
}




In [2]:
# Membersihkan teks
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text



In [3]:
# Mengonversi JSON ke DataFrame
def json_to_dataframe(intents):
    patterns = []
    tags = []
    responses = []

    for intent in intents["intents"]:
        for pattern in intent["patterns"]:
            patterns.append(clean_text(pattern))
            tags.append(intent["tag"])
        responses.append({intent["tag"]: intent["responses"]})

    data = pd.DataFrame({"patterns": patterns, "tags": tags})
    return data, responses



In [4]:
# Kelas chatbot
class HearingChatbot:
    def __init__(self, intents, max_words=1000, max_len=20):
        self.intents = intents
        self.max_words = max_words
        self.max_len = max_len
        self.tokenizer = None
        self.model = None
        self.labels = []
        self.responses = {}

        self.prepare_data()
        self.build_model()

    def prepare_data(self):
        # Mengonversi JSON ke DataFrame
        data, responses = json_to_dataframe(self.intents)

        # Menyimpan responses untuk digunakan nanti
        for response in responses:
            self.responses.update(response)

        # Tokenisasi
        self.tokenizer = Tokenizer(num_words=self.max_words, oov_token="<OOV>")
        self.tokenizer.fit_on_texts(data["patterns"])

        # Mengubah text menjadi sequences
        sequences = self.tokenizer.texts_to_sequences(data["patterns"])
        padded_sequences = pad_sequences(sequences, truncating='post', maxlen=self.max_len, padding='post')

        # Mengubah labels menjadi numeric
        self.labels = data["tags"].unique().tolist()
        self.label_encoder = {label: i for i, label in enumerate(self.labels)}
        self.reverse_label_encoder = {i: label for i, label in enumerate(self.labels)}

        numeric_labels = np.array([self.label_encoder[tag] for tag in data["tags"]])

        # Menyiapkan data training
        self.train_x = padded_sequences
        self.train_y = tf.keras.utils.to_categorical(numeric_labels, num_classes=len(self.labels))

    def build_model(self):
        # Membuat model
        self.model = tf.keras.Sequential([
            tf.keras.layers.Embedding(self.max_words, 16, input_length=self.max_len),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(len(self.labels), activation='softmax')
        ])

        # Compile model
        self.model.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

        # Train model
        self.model.fit(self.train_x, self.train_y, epochs=500, verbose=0)

    def predict(self, text):
        # Preprocessing input
        cleaned_text = clean_text(text)
        sequence = self.tokenizer.texts_to_sequences([cleaned_text])

        # Jika tidak ada kata yang dikenali
        if not sequence[0]:
            return "Sorry, I couldn't understand that. Please try asking in a different way."

        padded = pad_sequences(sequence, maxlen=self.max_len, padding='post')

        # Prediksi
        pred = self.model.predict(padded, verbose=0)
        confidence = np.max(pred)  # Mendapatkan confidence tertinggi
        predicted_class = self.reverse_label_encoder[np.argmax(pred)]

        # Jika confidence rendah, keluarkan respons default
        if confidence < 0.3:
            return "Sorry, I don't understand your question. Could you rephrase it or type 'Info' for options?"

        # Return random response untuk class yang diprediksi
        return np.random.choice(self.responses[predicted_class])



In [5]:
# Function untuk testing chatbot
def test_chatbot():
    chatbot = HearingChatbot(intents)

    print("Hearity chatbot is ready to assist.! (Type 'quit' to exit.)")

    while True:
        user_input = input("You: ")

        if user_input.lower() == 'quit':
            print("Thank you for using our service!")
            break

        response = chatbot.predict(user_input)
        print("Bot:", response)

if __name__ == "__main__":
    test_chatbot()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Hearity chatbot is ready to assist.! (Type 'quit' to exit.)
You: Hai
Bot: Hello! Welcome to the Hearity application. We do not understand your question yet. Please type 'Info' to see quick questions.
You: info
Bot: Welcome to our service! Please type the following number to get an answer from the set of questions we have provided:
1. What is hearing test?
2. What is hearing loss?
3. Other Information
You: 1
Bot: A hearing test is a clinical evaluation that measures an individual's ability to hear sounds at various frequencies and volumes to assess hearing health.
You: hearing test
Bot: A hearing test is a clinical evaluation that measures an individual's ability to hear sounds at various frequencies and volumes to assess hearing health.
You: hearing loss
Bot: Hearing loss is a partial or total inability to hear sounds, which can result from age, noise exposure, genetics, or other medical conditions.
You: 2
Bot: Hearing loss is a partial or total inability to hear sounds, which can resu